## Importing libraries

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import os
import cv2
import math
import scipy
from scipy import ndimage
from tensorflow.python.framework import ops
from sklearn.model_selection import train_test_split
from tensorflow.python.saved_model import tag_constants
from tensorflow.python.tools.inspect_checkpoint import print_tensors_in_checkpoint_file

## Defining loss function

In [3]:
def loss_1(labels, logits,batch_size):
    labels = tf.to_float(labels)
    sum=tf.zeros([1,], dtype=tf.float32)
    for i in range(batch_size):
        cx_label = tf.divide(tf.add(labels[i][0],labels[i][1]),2)
        cy_label = tf.divide(tf.add(labels[i][2],labels[i][3]),2)
        cx_logits = tf.divide(tf.add(logits[i][0],logits[i][1]),2)
        cy_logits = tf.divide(tf.add(logits[i][2],logits[i][3]),2)
        c_dist=tf.add(tf.square(tf.subtract(cx_label, cx_logits)),tf.square(tf.subtract(cy_label, cy_logits)))
        bx_label=tf.subtract(labels[i][1],labels[i][0])
        bx_logits=tf.subtract(logits[i][1],logits[i][0])
        hy_label=tf.subtract(labels[i][3],labels[i][2])
        hy_logits=tf.subtract(logits[i][3],logits[i][2])
        b_diff=tf.square(tf.subtract(bx_label, bx_logits))
        h_diff=tf.square(tf.subtract(hy_label, hy_logits))
        loss=tf.add(tf.add(b_diff, h_diff), c_dist)
        sum=sum+loss
    return sum/batch_size

## The CNN model

In [4]:
def cnn_model_fn(features, labels, mode):
  """Model function for CNN."""
  # Input Layer
  input_layer = tf.reshape(features["x"], [-1, 480, 640, 3])

  # Convolutional Layer #1
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=8,
      use_bias=True,
      bias_initializer=tf.zeros_initializer(),
      kernel_size=[1, 1],
      padding="same",
      strides=(1, 1),
      activation=tf.nn.relu)

  # Pooling Layer #1
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=16,
      kernel_size=[1, 1],
      padding="same",
      strides=(1, 1),
      activation=tf.nn.relu)


  conv3 = tf.layers.conv2d(
      inputs=conv2,
      filters=16,
      kernel_size=[2, 2],
      padding="same",
      use_bias=True,
      bias_initializer=tf.zeros_initializer(),
      strides=(2, 2),
      activation=tf.nn.relu)


  conv4 = tf.layers.conv2d(
      inputs=conv3,
      filters=24,
      kernel_size=[1, 1],
      padding="same",
      strides=(1, 1),
      use_bias=True,
      bias_initializer=tf.zeros_initializer(),
      activation=tf.nn.relu)


  conv5 = tf.layers.conv2d(
      inputs=conv4,
      filters=24,
      kernel_size=[2, 2],
      padding="same",
      strides=(2, 2),
      use_bias=True,
      bias_initializer=tf.zeros_initializer(),
      activation=tf.nn.relu)


#       conv6 = tf.layers.conv2d(
#           inputs=conv5,
#           filters=48,
#           kernel_size=[1, 1],
#           padding="same",
#           strides=(1, 1),
#           use_bias=True,
#           bias_initializer=tf.zeros_initializer(),
#           activation=tf.nn.relu)

  pool3 = tf.layers.max_pooling2d(inputs=conv5, pool_size=[2, 2], strides=2)

  conv7 = tf.layers.conv2d(
      inputs=pool3,
      filters=48,
      kernel_size=[2, 2],
      padding="same",
      strides=(2, 2),
      activation=tf.nn.relu,
      use_bias=True,
      bias_initializer=tf.zeros_initializer())

  conv8 = tf.layers.conv2d(
      inputs=conv7,
      filters=24,
      kernel_size=[1, 1],
      padding="same",
      strides=(1, 1),
      activation=tf.nn.relu,
      use_bias=True,
      bias_initializer=tf.zeros_initializer())

  conv9 = tf.layers.conv2d(
      inputs=conv8,
      filters=48,
      kernel_size=[1, 1],
      padding="same",
      strides=(1, 1),
      activation=tf.nn.relu,
      use_bias=True,
      bias_initializer=tf.zeros_initializer())

  conv10 = tf.layers.conv2d(
      inputs=conv9,
      filters=48,
      kernel_size=[2, 2],
      padding="same",
      strides=(2, 2),
      activation=tf.nn.relu,
      use_bias=True,
      bias_initializer=tf.zeros_initializer())

  conv11 = tf.layers.conv2d(
      inputs=conv10,
      filters=48,
      kernel_size=[1, 1],
      padding="same",
      strides=(1, 1),
      activation=tf.nn.relu,
      use_bias=True,
      bias_initializer=tf.zeros_initializer())


  conv12 = tf.layers.conv2d(
      inputs=conv11,
      filters=48,
      kernel_size=[1, 1],
      padding="same",
      strides=(1, 1),
      activation=tf.nn.relu,
      use_bias=True,
      bias_initializer=tf.zeros_initializer())
    
  # Dense Layer
  conv12_flat = tf.reshape(conv12, [-1, 3840 ])
    
  dense = tf.layers.dense(inputs=conv12_flat, units=120, activation=tf.nn.relu)

  dropout = tf.layers.dropout(
      inputs=dense, rate=0.1, training=mode == tf.estimator.ModeKeys.TRAIN)

  # Logits Layer
  logits = tf.layers.dense(inputs=dropout, units=4)

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "co-ordinates": tf.multiply(x = tf.convert_to_tensor(np.array([255.00],dtype = 'float32')), y = logits, name="softmax_tensor")
  }

  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  #loss = tf.losses.mean_squared_error(labels, logits)
  loss = loss_1(labels,logits,batch_size=10)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.AdamOptimizer(learning_rate=0.00001)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "iou": tf.metrics.mean_iou(
          labels,logits,4)
  }

  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

##  1) Training the CNN model on the training dataset

In [5]:

#loading the path of the image location
path = 'D:\Flipkart\im\images'

#listing the name of the images
listing = (os.listdir(path))

#reading the 'train.csv'
df = pd.read_csv('D:\Flipkart\\training.csv')

#setting the index with column 'image_name' for easier access
df = df.set_index(keys='image_name',drop= True)

#some list and dicts for further storage and processing
true_listing = []
y_values = []
dict_={}

for i in listing:
    try:
        dict_[i] = pd.Series.tolist(df.loc[i])
        y_values.append(np.array(pd.Series.tolist(df.loc[i])))
        true_listing.append(i)
    except KeyError:
 
        continue
for k in range(0,13800,200):
    print(k)
#reading the images in the list_ for training

    j=0

    list_ = []
    for i in true_listing[k:]:
        try:
            list_.append(np.array(Image.open(path + '/' + i)))
            j+=1
            print(j, end=', ')
            if j>199:
                break
        except KeyError:
            continue
    
    #converting the list of arrays in to a single array
    #X_train values
    for i in range(len(list_)):
        list_[i] = list_[i].reshape(1,list_[i].shape[0],list_[i].shape[1],list_[i].shape[2])
    
    #Y_train values
    y_values_ = y_values[k:k+200]    
    for i in range(len(list_)):
        y_values_[i] = y_values_[i].reshape(1,y_values_[i].shape[0],) 
  
    ds_X = np.concatenate((list_[:]),axis = 0)
    ds_Y = np.concatenate((y_values_[:]),axis=0)
    X_train, X_test, Y_train, Y_test = train_test_split(ds_X, ds_Y, test_size=0.1, random_state=42)
    
    
    #idx = 15
    #plt.imshow(X_train[idx])
    #plt.plot(Y_train[idx][0],Y_train[idx][2],Y_train[idx][0],Y_train[idx][3],marker = 'o')
    #plt.plot(Y_train[idx][1],Y_train[idx][3],Y_train[idx][1],Y_train[idx][2],marker = 'o')
    
    
    X_train = X_train/255
    X_test = X_test/255
    Y_train = Y_train/255
    Y_test = Y_test/255
    Y_train = np.squeeze(Y_train)
    Y_test = np.squeeze(Y_test)
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    Y_train = Y_train.astype('float32')
    Y_test = Y_test.astype('int32')
          
    # Create the Estimator
    mnist_classifier = tf.estimator.Estimator(
        model_fn=cnn_model_fn, model_dir='D:\\reg_training_1')
    
    # Set up logging for predictions
    tensors_to_log = {"co-ordinates": "softmax_tensor"}
    
    logging_hook = tf.train.LoggingTensorHook(
        tensors=tensors_to_log, every_n_iter=50)
    
    # Train the model
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": X_train},
        y=Y_train,
        batch_size=10,     
        num_epochs=10,
        shuffle=True)
    
    # train one step and display the probabilties
    mnist_classifier.train(
        input_fn=train_input_fn,
        steps=1,
        hooks=[logging_hook])
    
    mnist_classifier.train(input_fn=train_input_fn, steps=200)
    
    #for evaluation of the trained model
    eval_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": X_test},
        y=Y_test,
        num_epochs=1,
        shuffle=False)
    #X = np.expand_dims(X_test[0],axis=0)
    
    
    eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
    print(eval_results)
    

0
1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'D:\\reg_training_1', '

## 2) Training the CNN model on the vertically flipped training dataset

In [4]:
df_vflip = pd.read_csv('D:\Flipkart\\vfliptraining.csv')

for k in range(0,13800,200):
    print(k)
    #for flipped images uncomment for activation
    list_ = []
    y_values_ = []
    for i in range(k,k+200,1):
        first=np.array(df_vflip)[i]
        list_.append(cv2.flip(cv2.imread(path+'\\'+first[0]),0))
        y_values_.append(first[1:])
        if i == 199:
            break
    #print(len(list_))

    for i in range(len(list_)):
       
        list_[i] = list_[i].reshape(1,list_[i].shape[0],list_[i].shape[1],list_[i].shape[2])
    
    #Y_train values
 
    for i in range(len(y_values_)):
        y_values_[i] = y_values_[i].reshape(1,y_values_[i].shape[0],) 
        
    ds_X = np.concatenate((list_[:]),axis = 0)
    ds_Y = np.concatenate((y_values_[:]),axis=0)
    X_train, X_test, Y_train, Y_test = train_test_split(ds_X, ds_Y, test_size=0.1, random_state=42)
    
    
    #idx = 15
    #plt.imshow(X_train[idx])
    #plt.plot(Y_train[idx][0],Y_train[idx][2],Y_train[idx][0],Y_train[idx][3],marker = 'o')
    #plt.plot(Y_train[idx][1],Y_train[idx][3],Y_train[idx][1],Y_train[idx][2],marker = 'o')
    
    
    X_train = X_train/255
    X_test = X_test/255
    Y_train = Y_train/255
    Y_test = Y_test/255
    Y_train = np.squeeze(Y_train)
    Y_test = np.squeeze(Y_test)
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    Y_train = Y_train.astype('float32')
    Y_test = Y_test.astype('int32')
          
    # Create the Estimator
    mnist_classifier = tf.estimator.Estimator(
        model_fn=cnn_model_fn, model_dir="D:\\reg_training_1")
    
    # Set up logging for predictions
    tensors_to_log = {"co-ordinates": "softmax_tensor"}
    
    logging_hook = tf.train.LoggingTensorHook(
        tensors=tensors_to_log, every_n_iter=50)
    
    # Train the model
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": X_train},
        y=Y_train,
        batch_size=10,     
        num_epochs=5,
        shuffle=True)
    
    # train one step and display the probabilties
    mnist_classifier.train(
        input_fn=train_input_fn,
        steps=1,
        hooks=[logging_hook])
    
    mnist_classifier.train(input_fn=train_input_fn, steps=200)
    
    #for evaluation of the trained model
    eval_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": X_test},
        y=Y_test,
        num_epochs=1,
        shuffle=False)
    #X = np.expand_dims(X_test[0],axis=0)
    
    
    eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
    print(eval_results)


0
200
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'D:\\reg_training_1', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000213383C9198>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input 

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from D:\reg_training_1\model.ckpt-12671
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 12671 into D:\reg_training_1\model.ckpt.
INFO:tensorflow:co-ordinates = [[ 82.54155  573.8498   116.13881  361.041   ]
 [227.19408  468.5809   101.10268  465.15515 ]
 [151.73155  494.8076    66.018555 403.8191  ]
 [ 96.41232  386.40247   53.53523  387.48285 ]
 [152.62105  564.57324    3.653289 362.50925 ]
 [ 77.67818  688.38385  148.02968  443.5351  ]
 [ 79.634735 538.8488   144.39366  340.0908  ]
 [198.83867  555.40814   82.457985 431.42526 ]
 [152.01994  416.68686   84.658714 326.04056 ]
 [ 22.620026 519.9648   167.20572  365.7109  ]]
INFO:tensorflow:loss = 0.17175107, step = 12671
INFO:tensorflow:Saving checkpoints for 12672 into D:\reg

INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-02-11-13:10:59
INFO:tensorflow:Saving dict for global step 12944: global_step = 12944, iou = 0.8160395, loss = 0.55561495
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 12944: D:\reg_training_1\model.ckpt-12944
{'iou': 0.8160395, 'loss': 0.55561495, 'global_step': 12944}
1000
200
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'D:\\reg_training_1', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_s

INFO:tensorflow:loss = 0.23309977, step = 13126
INFO:tensorflow:Saving checkpoints for 13127 into D:\reg_training_1\model.ckpt.
INFO:tensorflow:Loss for final step: 0.23309977.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from D:\reg_training_1\model.ckpt-13127
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 13127 into D:\reg_training_1\model.ckpt.
INFO:tensorflow:loss = 0.1734863, step = 13127
INFO:tensorflow:Saving checkpoints for 13217 into D:\reg_training_1\model.ckpt.
INFO:tensorflow:Loss for final step: 0.16824703.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-02-11-13:14:16
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from D:\reg_training_1\model.ckpt-13217
INFO:te

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from D:\reg_training_1\model.ckpt-13399
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 13399 into D:\reg_training_1\model.ckpt.
INFO:tensorflow:co-ordinates = [[ 24.980053  487.65533    78.44129   400.2798   ]
 [ 31.122952  576.905      54.186466  315.42892  ]
 [ 63.61586   498.4692    127.25389   293.7296   ]
 [135.28566   551.25995   122.47557   405.5885   ]
 [174.76453   633.47864   153.21297   377.5581   ]
 [  0.3716495 581.9538     92.59377   424.0743   ]
 [ 89.6615    392.0819     71.87674   298.68668  ]
 [ 99.39213   453.81155    42.123943  278.75922  ]
 [ 92.28804   430.1127     80.30371   348.6728   ]
 [ 95.414764  561.43054   130.08891   298.2718   ]]
INFO:tensorflow:loss = 0.5569939, step = 13399
INFO:tensorflow:S

...

INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-02-11-14:01:26
INFO:tensorflow:Saving dict for global step 17767: global_step = 17767, iou = 0.93086416, loss = 0.7117619
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 17767: D:\reg_training_1\model.ckpt-17767
{'iou': 0.93086416, 'loss': 0.7117619, 'global_step': 17767}
11600
200
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'D:\\reg_training_1', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_

INFO:tensorflow:loss = 0.29885754, step = 17949
INFO:tensorflow:Saving checkpoints for 17950 into D:\reg_training_1\model.ckpt.
INFO:tensorflow:Loss for final step: 0.29885754.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from D:\reg_training_1\model.ckpt-17950
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 17950 into D:\reg_training_1\model.ckpt.
INFO:tensorflow:loss = 0.14233325, step = 17950
INFO:tensorflow:Saving checkpoints for 18040 into D:\reg_training_1\model.ckpt.
INFO:tensorflow:Loss for final step: 0.1327338.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-02-11-14:04:13
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from D:\reg_training_1\model.ckpt-18040
INFO:te

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from D:\reg_training_1\model.ckpt-18222
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 18222 into D:\reg_training_1\model.ckpt.
INFO:tensorflow:co-ordinates = [[119.41694  615.74036  106.8609   379.36142 ]
 [123.28378  582.68097  107.2784   409.82764 ]
 [ 66.61054  482.01044  142.56491  389.78613 ]
 [247.27434  541.234     67.19357  417.57825 ]
 [146.33418  631.29504   67.05984  465.0469  ]
 [203.4491   437.38406   52.365334 374.7407  ]
 [ 75.44037  608.7651    77.07859  448.0781  ]
 [145.43164  492.86057   92.4754   390.55692 ]
 [100.4629   535.131     68.7275   511.95004 ]
 [142.73299  459.23584  121.33518  428.564   ]]
INFO:tensorflow:loss = 0.23799239, step = 18222
INFO:tensorflow:Saving checkpoints for 18223 into D:\reg

INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-02-11-14:08:47
INFO:tensorflow:Saving dict for global step 18495: global_step = 18495, iou = 0.8300505, loss = 0.7706589
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 18495: D:\reg_training_1\model.ckpt-18495
{'iou': 0.8300505, 'loss': 0.7706589, 'global_step': 18495}
13200
200
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'D:\\reg_training_1', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_se

INFO:tensorflow:loss = 0.22659563, step = 18677
INFO:tensorflow:Saving checkpoints for 18678 into D:\reg_training_1\model.ckpt.
INFO:tensorflow:Loss for final step: 0.22659563.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from D:\reg_training_1\model.ckpt-18678
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 18678 into D:\reg_training_1\model.ckpt.
INFO:tensorflow:loss = 0.19464399, step = 18678
INFO:tensorflow:Saving checkpoints for 18768 into D:\reg_training_1\model.ckpt.
INFO:tensorflow:Loss for final step: 0.37958235.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-02-11-14:11:31
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from D:\reg_training_1\model.ckpt-18768
INFO:t

## 3) Training the CNN model on the horizontally flipped training dataset

In [5]:
df_hflip = pd.read_csv('D:\Flipkart\\hfliptraining.csv')

for k in range(0,13800,200):
    print(k)
    #for flipped images uncomment for activation
    list_ = []
    y_values_ = []
    for i in range(k,k+200,1):
        first=np.array(df_hflip)[i]
        list_.append(cv2.flip(cv2.imread(path+'\\'+first[0]),1))
        y_values_.append(first[1:])
        if i == 199:
            break

    for i in range(len(list_)):
       
        list_[i] = list_[i].reshape(1,list_[i].shape[0],list_[i].shape[1],list_[i].shape[2])
    
    #Y_train values
 
    for i in range(len(y_values_)):
        y_values_[i] = y_values_[i].reshape(1,y_values_[i].shape[0],) 
        
    ds_X = np.concatenate((list_[:]),axis = 0)
    ds_Y = np.concatenate((y_values_[:]),axis=0)
    X_train, X_test, Y_train, Y_test = train_test_split(ds_X, ds_Y, test_size=0.1, random_state=42)
    
    X_train = X_train/255
    X_test = X_test/255
    Y_train = Y_train/255
    Y_test = Y_test/255
    Y_train = np.squeeze(Y_train)
    Y_test = np.squeeze(Y_test)
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    Y_train = Y_train.astype('float32')
    Y_test = Y_test.astype('int32')
          
    # Create the Estimator
    mnist_classifier = tf.estimator.Estimator(
        model_fn=cnn_model_fn, model_dir="D:\\reg_training_1")
    
    # Set up logging for predictions
    tensors_to_log = {"co-ordinates": "softmax_tensor"}
    
    logging_hook = tf.train.LoggingTensorHook(
        tensors=tensors_to_log, every_n_iter=50)
    
    # Train the model
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": X_train},
        y=Y_train,
        batch_size=10,     
        num_epochs=5,
        shuffle=True)
    
    # train one step and display the probabilties
    mnist_classifier.train(
        input_fn=train_input_fn,
        steps=1,
        hooks=[logging_hook])
    
    mnist_classifier.train(input_fn=train_input_fn, steps=200)
    
    #for evaluation of the trained model
    eval_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": X_test},
        y=Y_test,
        num_epochs=1,
        shuffle=False)
    #X = np.expand_dims(X_test[0],axis=0)
    
    
    eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
    print(eval_results)
    

0
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'D:\\reg_training_1', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000002130DE03D68>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:

INFO:tensorflow:loss = 0.51294523, step = 18950
INFO:tensorflow:Saving checkpoints for 18951 into D:\reg_training_1\model.ckpt.
INFO:tensorflow:Loss for final step: 0.51294523.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from D:\reg_training_1\model.ckpt-18951
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 18951 into D:\reg_training_1\model.ckpt.
INFO:tensorflow:loss = 0.29972383, step = 18951
INFO:tensorflow:Saving checkpoints for 19041 into D:\reg_training_1\model.ckpt.
INFO:tensorflow:Loss for final step: 0.19148378.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-02-11-14:14:30
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from D:\reg_training_1\model.ckpt-19041
INFO:t

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from D:\reg_training_1\model.ckpt-19223
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 19223 into D:\reg_training_1\model.ckpt.
INFO:tensorflow:co-ordinates = [[111.475334 463.42096   23.704758 308.7546  ]
 [142.84468  444.87866   13.41913  416.64178 ]
 [160.87735  470.67523   81.89042  411.74585 ]
 [ 78.09036  453.5045     5.116951 385.09393 ]
 [ 55.363472 496.41757  184.00557  321.93182 ]
 [ 85.156555 665.9512   178.67853  443.26968 ]
 [102.06734  580.8119   -13.543909 530.85016 ]
 [120.878105 460.7957    95.57053  322.36063 ]
 [108.687325 484.40198   94.56185  321.03015 ]
 [153.18242  420.90805   96.107704 307.53268 ]]
INFO:tensorflow:loss = 0.30006695, step = 19223
INFO:tensorflow:Saving checkpoints for 19224 into D:\reg

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from D:\reg_training_1\model.ckpt-19496
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-02-11-14:19:17
INFO:tensorflow:Saving dict for global step 19496: global_step = 19496, iou = 0.78300864, loss = 0.70721847
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 19496: D:\reg_training_1\model.ckpt-19496
{'iou': 0.78300864, 'loss': 0.70721847, 'global_step': 19496}
1600
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'D:\\reg_training_1', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribut

INFO:tensorflow:loss = 0.11333574, step = 19678
INFO:tensorflow:Saving checkpoints for 19679 into D:\reg_training_1\model.ckpt.
INFO:tensorflow:Loss for final step: 0.11333574.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from D:\reg_training_1\model.ckpt-19679
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 19679 into D:\reg_training_1\model.ckpt.
INFO:tensorflow:loss = 0.35330638, step = 19679
INFO:tensorflow:Saving checkpoints for 19769 into D:\reg_training_1\model.ckpt.
INFO:tensorflow:Loss for final step: 0.20197628.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-02-11-14:22:02
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from D:\reg_training_1\model.ckpt-19769
INFO:t

...

INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-02-11-15:02:21
INFO:tensorflow:Saving dict for global step 23864: global_step = 23864, iou = 0.80215055, loss = 0.77795297
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 23864: D:\reg_training_1\model.ckpt-23864
{'iou': 0.80215055, 'loss': 0.77795297, 'global_step': 23864}
11200
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'D:\\reg_training_1', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_se

INFO:tensorflow:loss = 0.17050438, step = 24046
INFO:tensorflow:Saving checkpoints for 24047 into D:\reg_training_1\model.ckpt.
INFO:tensorflow:Loss for final step: 0.17050438.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from D:\reg_training_1\model.ckpt-24047
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 24047 into D:\reg_training_1\model.ckpt.
INFO:tensorflow:loss = 0.09202253, step = 24047
INFO:tensorflow:Saving checkpoints for 24137 into D:\reg_training_1\model.ckpt.
INFO:tensorflow:Loss for final step: 0.4484404.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-02-11-15:05:00
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from D:\reg_training_1\model.ckpt-24137
INFO:te

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from D:\reg_training_1\model.ckpt-24319
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 24319 into D:\reg_training_1\model.ckpt.
INFO:tensorflow:co-ordinates = [[133.38095   506.86444   157.69173   354.13852  ]
 [163.96138   575.53156    98.347595  365.8529   ]
 [ 54.253597  543.0115     34.281574  395.58636  ]
 [ 94.43941   499.3633    130.87056   300.44745  ]
 [ 88.32748   607.0628    170.67755   340.1391   ]
 [ -5.8905973 608.167     151.45476   223.8849   ]
 [ 21.031027  513.39667    90.41832   330.39517  ]
 [ 95.17982   567.9496     43.549896  353.64633  ]
 [122.30439   472.35767    48.88187   411.0447   ]
 [120.950806  519.4842    172.2439    367.48758  ]]
INFO:tensorflow:loss = 0.19258241, step = 24319
INFO:tensorflow:

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from D:\reg_training_1\model.ckpt-24592
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-02-11-15:09:22
INFO:tensorflow:Saving dict for global step 24592: global_step = 24592, iou = 0.8194444, loss = 0.7593098
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 24592: D:\reg_training_1\model.ckpt-24592
{'iou': 0.8194444, 'loss': 0.7593098, 'global_step': 24592}
12800
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'D:\\reg_training_1', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute':

INFO:tensorflow:loss = 0.19112451, step = 24774
INFO:tensorflow:Saving checkpoints for 24775 into D:\reg_training_1\model.ckpt.
INFO:tensorflow:Loss for final step: 0.19112451.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from D:\reg_training_1\model.ckpt-24775
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 24775 into D:\reg_training_1\model.ckpt.
INFO:tensorflow:loss = 0.16553786, step = 24775
INFO:tensorflow:Saving checkpoints for 24865 into D:\reg_training_1\model.ckpt.
INFO:tensorflow:Loss for final step: 0.091728866.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-02-11-15:12:06
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from D:\reg_training_1\model.ckpt-24865
INFO:

## Getting predition out of the trained model on the test images

In [80]:
#Loading the test set and getting the results

df1 = pd.read_csv('D:\Flipkart\\test.csv') #Dataframe containing the test-data
test_list=np.array(df1)[:, 0] #Extracting the names of the images


for i in range(0, len(test_list), 50): #Predicting 
    print(i)
    test_list1=test_list[i:i+50]
    batch=len(test_list1)
    input_X=np.zeros((batch, 480, 640, 3))
    
    for j in range(len(test_list1)):
        input_X[j, :, :, :]=cv2.imread(path + '\\' + test_list1[j])
    
    input_X=input_X.astype('float32')/255
    
    pred_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": input_X},
    shuffle=False)

    #calling the prediction function
    predict = mnist_classifier.predict(input_fn = pred_input_fn)
    
    #unwrapping the predict iterator
    cls_pred = np.array(list(predict))
    pred=np.zeros((batch, 4))
    
    for j in range(len(test_list1)):
        pred[j, :]=cls_pred[j]['co-ordinates']
        if pred[j, 0]<0:
            pred[j, 0]=1
        if pred[j, 1]>640:
            pred[j, 1]=639
        if pred[j, 2]<0:
            pred[j, 2]=1
        if pred[j ,3]>480:
            pred[j, 3]=479
            
    for j in range(len(test_list1)):
        df1.loc[i+j, 'x1']=pred[j][0]
        df1.loc[i+j, 'x2']=pred[j][1]
        df1.loc[i+j, 'y1']=pred[j][2]
        df1.loc[i+j, 'y2']=pred[j][3]
    
df1.to_csv('D:\Flipkart\\test_updated.csv',index=False)

0
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from D:\reg_training_1\model.ckpt-25228
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
50
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from D:\reg_training_1\model.ckpt-25228
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
100
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from D:\reg_training_1\model.ckpt-25228
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
150
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from D:\reg_training_1\model.c

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from D:\reg_training_1\model.ckpt-25228
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
1100
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from D:\reg_training_1\model.ckpt-25228
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
1150
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from D:\reg_training_1\model.ckpt-25228
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
1200
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from D:\reg_training_1\model.ckpt-25228
INFO:tensorflow:Runnin

2550
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from D:\reg_training_1\model.ckpt-25228
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
2600
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from D:\reg_training_1\model.ckpt-25228
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
2650
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from D:\reg_training_1\model.ckpt-25228
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
2700
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from D:\reg_training_1\

INFO:tensorflow:Done running local_init_op.
4050
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from D:\reg_training_1\model.ckpt-25228
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
4100
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from D:\reg_training_1\model.ckpt-25228
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
4150
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from D:\reg_training_1\model.ckpt-25228
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
4200
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:

INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
5550
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from D:\reg_training_1\model.ckpt-25228
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
5600
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from D:\reg_training_1\model.ckpt-25228
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
5650
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from D:\reg_training_1\model.ckpt-25228
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
5700
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflo

INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
7050
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from D:\reg_training_1\model.ckpt-25228
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
7100
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from D:\reg_training_1\model.ckpt-25228
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
7150
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from D:\reg_training_1\model.ckpt-25228
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
7200
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflo

INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
8550
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from D:\reg_training_1\model.ckpt-25228
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
8600
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from D:\reg_training_1\model.ckpt-25228
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
8650
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from D:\reg_training_1\model.ckpt-25228
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
8700
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflo

INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
10050
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from D:\reg_training_1\model.ckpt-25228
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
10100
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from D:\reg_training_1\model.ckpt-25228
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
10150
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from D:\reg_training_1\model.ckpt-25228
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
10200
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tenso

INFO:tensorflow:Restoring parameters from D:\reg_training_1\model.ckpt-25228
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
11550
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from D:\reg_training_1\model.ckpt-25228
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
11600
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from D:\reg_training_1\model.ckpt-25228
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
11650
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from D:\reg_training_1\model.ckpt-25228
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
11700
INFO:t